## Imports

In [31]:
import pandas as pd
import json

## Loading the data

In [ ]:
# Function taken from https://stackoverflow.com/questions/52795561/flattening-nested-json-in-pandas-data-frame.
def flatten_json(nested_json, exclude=['']):
    """Flatten json object with nested keys into a single level.
        Args:
            nested_json: A nested json object.
            exclude: Keys to exclude from output.
        Returns:
            The flattened json object if successful, None otherwise.
    """
    out = {}

    def flatten(x, name='', exclude=exclude):
        if type(x) is dict:
            for a in x:
                if a not in exclude: flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(nested_json)
    return out

In [44]:
# Loading sleep data from Oura json file available on https://cloud.ouraring.com/profile 
with open('/Users/user/Desktop/Msc AI RU/Internship/Code/oura_json/oura_sleep_2022-12-06T10-07-33.json') as f:
    data1 = json.load(f)

flattened_sleep = flatten_json(data1['sleep'])
flattened_json = json.dumps(flattened_sleep)

with open('oura_json/flattened_sleep.json', 'w') as outfile:
    outfile.write(flattened_json)

flattened_df = pd.read_json("oura_json/flattened_sleep.json", orient ='index')

# Loading readiness data from Oura json file available on https://cloud.ouraring.com/profile 
with open('/Users/user/Desktop/Msc AI RU/Internship/Code/oura_json/oura_daily-readiness_2022-12-06T10-07-44.json') as f:
    data2 = json.load(f)

readiness_data = data2['daily_readiness']
readiness_json = json.dumps(readiness_data)

with open('oura_json/readiness_data.json', 'w') as outfile:
    outfile.write(readiness_json)

readiness_df = pd.read_json("oura_json/readiness_data.json")

In [ ]:
# df = pd.read_json(flattened_sleep, orient ='index')
# sleep_data = data1['sleep']
# readiness_json = json.dumps(sleep_data)
# df = pd.DataFrame(sleep_data)
# normalized_df = json_normalize(data1['sleep'])
# print(df)
# with open('oura_json/sleep_data.json', 'w') as outfile:
#     outfile.write(sleep_json)
# sleep_df = pd.read_json('oura_json/sleep_data.json')

In [93]:
with open('/Users/user/Desktop/Msc AI RU/Internship/Code/oura_json/oura_sleep_2022-12-06T10-07-33.json') as f:
    data1 = json.load(f)

df = pd.json_normalize(data1['sleep'], max_level=2)

bedtime_end
bedtime_start
day
period
segment_state
time_in_bed
type
sleep_phase_5_min
restless
timezone
bedtime_start_delta
bedtime_end_delta
midpoint_at_delta
contributors.total_sleep
contributors.deep_sleep
contributors.rem_sleep
contributors.efficiency
contributors.latency
contributors.restfulness
contributors.timing
heart_rate.interval
heart_rate.items
heart_rate.timestamp
hrv.interval
hrv.items
hrv.timestamp
average_breath
average_breath_variation
average_heart_rate
average_hrv
awake_time
deep_sleep_duration
efficiency
got_ups
latency
light_sleep_duration
lowest_heart_rate
lowest_heart_rate_time_offset
movement_30_sec
rem_sleep_duration
restless_periods
score
sleep_midpoint
total_sleep_duration
wake_ups
readiness.contributors.activity_balance
readiness.contributors.body_temperature
readiness.contributors.hrv_balance
readiness.contributors.previous_day_activity
readiness.contributors.previous_night
readiness.contributors.recovery_index
readiness.contributors.resting_heart_rate
read

In [72]:
# Opening the file
file = open('/Users/user/Desktop/Msc AI RU/Internship/Code/oura_json/oura_sleep_2022-12-06T10-07-33.json')

# Loading the file into a workable dataframe
data1 = json.load(file)
df = pd.DataFrame()

counter = 0 
for i in data1['sleep']:
    flattened_df = pd.DataFrame()
    # if counter == 0:
    #     print(i)
    counter+=1
    flattened_row = flatten_json(i)
    flattened_df = pd.DataFrame(flattened_row.items())

    # flattened_list = list(flattened_row)
    # flattened_json = json.dumps(flattened_row)
    # print(type(flattened_row))
    # print(type(flattened_json))
    if counter == 5:
        print(flattened_df)
    #     print(flattened_json)
    df.append(flattened_df)

print(df)


                            0       1
0              average_breath  16.625
1    average_breath_variation   2.875
2          average_heart_rate   55.66
3                 average_hrv      70
4                  awake_time    2490
..                        ...     ...
231                  restless       7
232                  timezone     120
233       bedtime_start_delta    -663
234         bedtime_end_delta   26517
235         midpoint_at_delta   12447

[236 rows x 2 columns]
Empty DataFrame
Columns: []
Index: []


In [ ]:
wakeup_df = pd.read_csv ('questionnaire_data/raw/Wakeup_3_v1_raw.csv')

bedtime_df = pd.read_csv ('questionnaire_data/raw/Bedtime_3_v1_raw.csv')